In [113]:
################################################################################################
################ CMSC320 Project 4: Interactive Data Maps
################ Group_Students: Ang Zhou, Han Yang Shen, Jixiang Fan
################################################################################################

# In this mini project, we create an interactive criminal data map in baltimore. We use 2 different markers to identify the criminal's gender:
# blue for male and red for female. Each popup of the criminal marker would contain the specific info about the criminal, including race,
# arrest date and brief description about the the charge. In order to make the map look more clean, we use the markercluster plugin to group
# the criminal events within district around. This would help people to see a general distribution of the criminal events in baltimore.

import folium
import requests
import pandas

#we extreact data from the following website and create a dataframe of the top 200 criminal events
arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table["race_new"] = arrest_table["sex"]
arrest_table["sex_new"] = arrest_table["race"]
arrest_table["race"] = arrest_table["race_new"]
arrest_table["sex"] = arrest_table["sex_new"]
arrest_table = arrest_table.drop('race_new', 1)
arrest_table = arrest_table.drop('sex_new', 1)

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

arrest_table=arrest_table.head(200)
arrest_table

,arrest,age,sex,race,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,M,B,01/01/2011,00:01,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,M,B,01/01/2011,00:01,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,M,B,01/01/2011,00:04,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,M,B,01/01/2011,00:05,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,M,B,01/01/2011,00:05,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358
6,11126932.0,29,M,B,01/01/2011,00:05,800 N Monroe St,79-Other,800 N Monroe St,1 5212,Handgun On Person || Handgun Violation,WESTERN,724.0,Midtown-Edmondson,"(39.2979815407, -76.6475113571)",39.297982,-76.647511
7,11126940.0,20,M,W,01/01/2011,00:05,5200 Moravia Rd,Unknown Offense,NaN,1 5200,Deadly Weapon-Int/Injure || Aggravated Assault,NORTHEASTERN,436.0,Frankford,"(39.3235271620, -76.5496555072)",39.323527,-76.549656
9,11127018.0,53,M,B,01/01/2011,00:15,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,1 1425,Reckless Endangerment || Hgv,NORTHWESTERN,614.0,Central Park Heights,"(39.3436773374, -76.6727297618)",39.343677,-76.672730
10,11127057.0,28,M,B,01/01/2011,00:15,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,1 1425,Reckless Endangerment || Hgv,NORTHWESTERN,614.0,Central Park Heights,"(39.3436773374, -76.6727297618)",39.343677,-76.672730
11,11126892.0,25,M,B,01/01/2011,00:20,2800 Violet Ave,79-Other,2800 Violet Av,1 5212,Handgun On Person || Hgv,NORTHWESTERN,612.0,Park Circle,"(39.3298283117, -76.6618069447)",39.329828,-76.661807


In [114]:
#brief view of the map of baltimore
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm

In [115]:
from folium import IFrame
from folium.plugins import MarkerCluster

#table form for popup for each marker
table = """
<!DOCTYPE html>
<html>
<head>
<style>
table {{
    width:100%;
}}
table, th, td {{
    border: 1px solid black;
    border-collapse: collapse;
}}
th, td {{
    padding: 5px;
    text-align: left;
}}
</style>
</head>
<body>

<table id="t01">
  <tr>
    <td>Gender</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>Race</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>Arrest Date</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>Description</td>
    <td>{}</td>
  </tr>
</table>
</body>
</html>
""".format

#marker cluster
marker_cluster = folium.MarkerCluster().add_to(map_osm)

for idx, row in arrest_table.iterrows():
    #if the criminal is a male, marker would be blue
    if (row['sex'] == 'M'):
        sex = str(row['sex'])
        race = str(row['race'])
        date = str(row['arrestDate'])
        chargeDecription = str(row['chargeDescription'])
        iframe = IFrame(table(sex, race, date, chargeDecription), width=500, height=200)
        popup = folium.Popup(iframe, max_width=500)
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=popup,
            icon=folium.Icon(color='blue', icon='info-sign')).add_to(marker_cluster)
    #if the criminal is a female, marker would be red
    elif (row['sex'] == 'F'):
        sex = str(row['sex'])
        race = str(row['race'])
        chargeDecription = str(row['chargeDescription'])
        iframe = IFrame(table(sex, race, date, chargeDecription), width=500, height=200)
        popup = folium.Popup(iframe, max_width=500)
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=popup,
            icon=folium.Icon(color='red', icon='info-sign')).add_to(marker_cluster)

map_osm